# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [ ]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# Exploration of Data

### Profile Data

<strong>profile.json</strong> file includes all demographic data for each customer

Its schema and explanation of each variable:

* age (int) - age of the customer
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

We can use `describe` method to get more stats about numerical columns

In [ ]:
profile.head()

It seems that there are multiple missing values in `gender` and `income` columns

In [ ]:
profile.describe()

It looks like we haven't formatted the datetime for `became_memeber_on`. Also there is outlier shows the maximum age is 118. We can address these issues more in the next satge for cleaning and transforming data

In [ ]:
print("Profile has {} rows and {} columns".format(profile.shape[0], profile.shape[1]))

In [ ]:
profile.isnull().any()

It seems that we have couple coulmns have missing values

The outliers in the age histogram and more and more cumstomers become the members year after year. and 2017 is the peak.

However, there is income cut off for each age groups as shown above. For people in 20s and 30s somewhere around 750K is a cut off for example. This indicates that the profile data is arbitrarily created. This may make prediciton simpler because user segmentation is clearler.

In [ ]:
pd.plotting.scatter_matrix(profile,  alpha=0.5, figsize=(10, 10), grid=True)

### Portfolio Data

<strong>portfolio.json</strong> - containing offer ids and meta data about each offer (duration, type, etc.)

Its schema and explanation of each variable:

* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

We can use `describe` method to get more stats about numerical columns

In [ ]:
portfolio.head()

In [ ]:
portfolio.describe()

In [ ]:
print("Portfolio has {} rows and {} columns".format(portfolio.shape[0], portfolio.shape[1]))

In [ ]:
portfolio["offer_type"].value_counts()

This is a small dataset regarding the offers, which includes 4 discounts, 2 bogos and 2 informational uniquely.

In [ ]:
list_channel = []
set([chan for channel in portfolio["channels"].tolist() for chan in channel])

There are 4 distinct channels

### Transcript Data

<strong>transcript.json</strong> - records for transactions, offers received, offers viewed, and offers completed

Its schema and explanation of each variable:

* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

We can use `describe` method to get more stats about numerical columns

In [ ]:
transcript.head()

In [ ]:
transcript.describe()

In [ ]:
print("Transcript has {} rows and {} columns".format(transcript.shape[0], transcript.shape[1]))

In [ ]:
transcript.event.value_counts()

In [ ]:
transcript["event"].value_counts().plot(kind="bar")
plt.show()

There is four types of events. Transaction, offer recieved and offer completed. They are reasonably balanced.



In [ ]:
received = transcript[transcript['event']=='offer received'].person.value_counts()
viewed = transcript[transcript['event']=='offer viewed'].person.value_counts()
completed = transcript[transcript['event']=='offer completed'].person.value_counts()

plt.hist([received, viewed, completed], alpha = 0.5, bins=10)
plt.legend(['offer_received', 'offer_viewed', 'offer_completed'])
plt.xlabel('count per user')
plt.ylabel('count')
plt.show()

More members tend to receive and viewed 4 to 5 offers. However, they are most likely complete the first 3 offers. Since most of people are curious about the offers at first. When more and more offers sent to the customers and expired in a validate time period, memebers may not be interested in as before. However, they do view and receive them.

# Data Cleanup and Transformation

### Profile Data

1. previously we noticed that `age` column has 118, which is over 2000 counts out of 17000 total rows. we simply replace them with overall mean. so does `income`.

2. hot-encoding on `gender` column

3. transform the `became_member_on` column to the duration of members as now in datetime format.

In [ ]:
def get_dummy_columns_profile(df, column_name, field_name):
    return df[column_name].apply(lambda x: 1 if x == field_name else 0)

In [ ]:
import datetime

def profile_cleanup_transform(df):
    p = df.copy()
    p['age'] = p['age'].apply(lambda x: np.nan if x == 118 else x)
    p['age'] = p['age'].fillna(p['age'].mean())
    
    p['income'] = p['income'].fillna(p['income'].mean())
    
    p["male"]   = get_dummy_columns_profile(p, 'gender', 'M')
    p["female"] = get_dummy_columns_profile(p, 'gender', 'F')
    p['not_spicify_gender'] = p['gender'].apply(lambda x: 1 if x != 'F' and x != 'M' else 0)
    p.drop(['gender'], axis=1, inplace=True)

    p['profile_id'] = p['id']
    p.drop(['id'], axis=1, inplace=True)
    p.set_index('profile_id',inplace=True)

    p['member_days'] = (datetime.datetime.today().date() - pd.to_datetime(p['became_member_on'], format='%Y%m%d').dt.date).dt.days
    p.drop(['became_member_on'], axis=1, inplace=True)
    return p

In [ ]:
new_profile = profile_cleanup_transform(profile)
new_profile.head()

### Portfolio Data

Since `channel` and `offer_type` are categorical columns, we can convert them into the dummy variable

In [ ]:
def get_dummy_columns_portfolio(df, column_name, field_name):
    return df[column_name].apply(lambda x: 1 if field_name in x else 0)

def portfolio_cleanup_transform(df):
    p = df.copy()
    p['web'] = get_dummy_columns_portfolio(p, "channels", "web")
    p['email'] = get_dummy_columns_portfolio(p, "channels", "email")
    p['mobile'] = get_dummy_columns_portfolio(p, "channels", "mobile")
    p['social'] = get_dummy_columns_portfolio(p, "channels", "social")
    p.drop(['channels'], axis=1, inplace=True)
    
    p['bogo'] = get_dummy_columns_portfolio(p, "offer_type", "bogo")
    p['discount'] = get_dummy_columns_portfolio(p, "offer_type", "discount")
    p['informational'] = get_dummy_columns_portfolio(p, "offer_type", "informational")
    p.drop(['offer_type'], axis=1, inplace=True)
    
    p['offer_id'] = p['id']
    p.set_index('offer_id', inplace=True)
    p.drop(['id'], axis=1, inplace=True)
    
    return p

In [ ]:
new_portfolio = portfolio_cleanup_transform(portfolio)
new_portfolio.head()

### Transcript Data

1. transform the `value` column and extract the `offer_id`
2. hot-encoding on the `event` column
3. drop off any irrelevant columns

In [ ]:
def transaction_cleanup_transform(df):
    p = df.copy()
    p['profile_id'] = p['person']
    p.drop(['person'], axis = 1, inplace = True)

    p['amount'] = p['value'].apply(lambda x: x['amount'])
    p.drop(['value'], axis = 1, inplace = True)
    p.drop(['time'], axis = 1, inplace = True)
    p.drop(['event'], axis = 1, inplace = True)
    
    p.reset_index(drop=True, inplace = True)
    return p

def offer_cleanup_transform(df):
    p = df.copy()
    p['profile_id'] = p['person']
    p.drop(['person'], axis = 1, inplace = True)
    
    p['offer_id'] = p['value'].apply(lambda x: x['offer id'] if 'offer id' in x.keys() else x['offer_id'] if 'offer_id' in x.keys() else np.nan)

    p['event'] = p['event'].apply(lambda x: x.replace(" ", "_"))                                 
    event_df = pd.get_dummies(p['event'])    
    p = pd.concat([p, event_df], axis=1)

    p.drop(['value', 'time', 'event'], axis = 1, inplace = True)
    p.reset_index(drop=True, inplace = True)
    return p

In [ ]:
transaction = transcript[transcript["event"] == "transaction"].copy()
new_transaction = transaction_cleanup_transform(transaction)
new_transaction.head()

In [ ]:
offer = transcript[transcript["event"] != "transaction"].copy()
new_offer = offer_cleanup_transform(offer)
new_offer.head()

###  Merge Data

In [ ]:
merged_data = pd.merge(new_offer, new_portfolio, on='offer_id')
merged_data = pd.merge(merged_data, new_profile, on='profile_id')

columns = merged_data.columns
columns.to_list()

offer_columns = ['offer_completed', 'offer_received', 'offer_viewed']

agg_func={}
for x in columns:
    if x in offer_columns:
        agg_func[x] = "sum"
    elif x not in ['profile_id', 'offer_id']:
        agg_func[x] = "mean"

grouped_data = merged_data.groupby(['profile_id', 'offer_id']).agg(agg_func)

grouped_data['offer_completed'] = grouped_data['offer_completed'].apply(lambda x: 1 if x > 0.0 else 0)

grouped_data.reset_index(inplace=True, drop=True)

offer_received = grouped_data['offer_received']
offer_viewed = grouped_data['offer_viewed']
offer_completed = grouped_data['offer_completed']

X = grouped_data.drop([
    'offer_completed', 
    'offer_received',
    'offer_viewed',], axis = 1)

In [ ]:
X.head()

### Normalize Grouped Data 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical = ['age', 'income', 'difficulty','duration', 'reward','member_days']

X[numerical] = scaler.fit_transform(X[numerical])
X.head()

### Split the Data

In [ ]:
from sklearn.model_selection import train_test_split

def split_data(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=1)
    return X_train, X_test, y_train, y_test


In [ ]:
X_train, X_test, y_train, y_test = split_data(X, offer_completed, 0.2)

# Benchmark Model

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import cross_val_score

In [ ]:
def train_and_prediction(model, X_train, y_train, X_test, y_test):
    print('Name: {}'.format(model.__class__.__name__))
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    score = model.score(X_test, y_test)
    print('Test score: {:.3f}'.format(score))
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy score: {:.3f}'.format(accuracy))

    probs = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, probs)
    print('AUC: %.2f' % auc)
    fpr, tpr, _ = roc_curve(y_test, probs)
    
    confusionMatrix = confusion_matrix(y_test, y_pred)
    
    return {'y_pred': y_pred, 'auc': auc, 'fpr': fpr, 'tpr': tpr, 'score': score, 'accuracy': accuracy, "confusion_matrix": confusionMatrix}

In [ ]:
from sklearn.dummy import DummyClassifier

model = DummyClassifier(strategy="most_frequent")

In [ ]:
train_and_prediction(model, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

models = []
models.append(DummyClassifier(strategy="most_frequent"))
models.append(SVC(probability=True))
models.append(MLPClassifier())
models.append(LogisticRegression())
models.append(KNeighborsClassifier())
models.append(SVC(kernel='linear',probability=True))
models.append(RandomForestClassifier())
models.append(AdaBoostClassifier())

In [ ]:
best_score = 0
best_method = ""

for model in models:
    result = train_and_prediction(model, X_train, y_train, X_test, y_test)
    fpr, tpr, auc = result['fpr'], result['tpr'], result['auc']
    plt.plot(fpr,tpr,label="{}, auc={}".format(model.__class__.__name__, str(auc)))
    
    if result['score'] > best_score:
        best_method = model.__class__.__name__
        best_score = result['score']

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

print('---------------------------------')
print('best_method：', best_method)
print('best_score：', best_score)